#S0: Data Source and Installing Packages

In [ ]:
#2.2M recipes from kaggle

# https://www.kaggle.com/datasets/paultimothymooney/recipenlg

In [8]:
#install dependencies

#pip = package installer for python, -q is quiet mode to suppress most dialogue

# !pip -q install transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu

#colab forgets about these so just reinstall each time
!pip -q install faiss-cpu
!pip -q install langchain-community
!pip -q install -U langchain-huggingface


#S1: Loading Pre-Trained Model

In [10]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/Phi-3-mini-128k-instruct"

# Load tokenizer
tok = AutoTokenizer.from_pretrained(model_name)

#device maps to gpu automatically as gpu > cpu for matrix math, but after usage limits, cpu used

#float16 uses half precision instead of full (32) which cuts memory use, speeds inference
#most of the time it doesnt matter but for computations it can

#its normal for this part to take a while to load

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16)

print("Model and tokenizer loaded successfully")
print(f"Running on {model.device}")





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully
Running on cuda:0


#S2: Loading Embeddings for Document Chunking

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

#to turn user queries into vectors as well as document chunks later, which are stored in vector db
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#example to verify it works
# print(embeddings.embed_query("Give me the recipe for a pizza"))

#S3: Connecting to Drive to Access Data

In [12]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#dataset of 2.2M recipes, ingredients (with named entity recognition), and directions
path = "/content/drive/My Drive/recipes.csv"

#just take the first 100k/2M rows so as to speed up the process of loading
df = pd.read_csv(path, nrows=100000)

#to verify it works
# print(df.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#S4: Data Cleaning to Feed to Model


In [13]:
#list columns for combine_recipe function, to see what is relevant
# print(df.columns)

#combining all info to embed and store to contain all relevant information for llm retrieval
def combine_recipe(row):
  info_per_row = f"Title: {row['title']}\nIngredients: {row['ingredients']}\nDirections: {row['directions']}"
  return info_per_row

#axis=1 = apply function to each row
df['info'] = df.apply(combine_recipe, axis=1)

#to verify it works
# print(df['info'][0]

#take only df[info] and convert to list for langchain's faiss, as it expects a list of strings
docs = df['info'].tolist()

#to verify it works
print(texts[0])


Device set to use cuda:0
/tmp/ipython-input-2899294009.py:67: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


#S5: Vector Database

In [ ]:
from langchain.vectorstores import FAISS

#its normal for this part to take a while to load

#takes each string in the "docs" list (each row of df['info']),
#turns each string into a vector embedding using the embedding model taken earlier, stores inside a faiss index
#user query is also vectorized, and the closest recipe representation in the vector space is taken as context for the llm response

db = FAISS.from_texts(texts=docs, embedding=embeddings)

#saves so re-embedding each time is not need, time efficient
db.save_local("faiss_index")
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


#S6: Model Settings and Langchain/HF Wrapping

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

#to put model in a langchain wrapper
from transformers import pipeline

#returns "k" vectors to query, here it retrieves 3 closest document chunk vectors, uses all as context for llm response
retriever = db.as_retriever(search_kwargs={"k":3})

#usually a device= arg in pipeline() but interferes here as i set gpu to auto earlier in pipeline so just uses that
#device=0 means use first gpu
#device=1 means use second gpu
#device=-1 means cpu

#tells the model its task, to generate text, and feeds in the neccessary information to do it

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    max_new_tokens=512,
    temperature=0.1, #creative but not too much, so it doesnt just riff about nonsense
    top_p=0.9,
    repetition_penalty=1.1,
    eos_token_id=tok.eos_token_id
)

#pipe is a huggingface object and this wraps it in langchain's llm interface
#so now langchain can call invoke(), .predict() or use inside chains like RetrievalQA
#wrapping is neccessary because hf pipelines return generated text in a list-like format and langchain expects plain strings, so the wrapper just converts it
llm = HuggingFacePipeline(pipeline=pipe)
# llm.pipeline_kwargs = {"stop": ["(END OF RECIPE)"]}

#S7: System Instructions for LLM

In [16]:
from langchain.prompts import PromptTemplate

template = """
You are a recipe assistant.
Output the recipe in this format:

Title: <title>
Ingredients:
- ...
Directions:
1. ...
2. ...
(END OF RECIPE)

Recipes:
{context}

Question: {question}

Answer:
"""

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt})


#S8: Interactive Display

In [ ]:
while True:

  #preferred that the user asks about food to test the implementation of RAG, but one can ask about anything
  query = input("Query: ")
  result = qa.invoke({"query": query})

  raw_output = result["result"]

  if "Answer:" in raw_output:
    answer = raw_output.split("Answer:")[-1].strip()
  else:
    answer = raw_output.strip()

  #result is a dictionary with keys named query, result, and source_documents
  print("\nAnswer:", answer, "\n")


Query: pizza recipe

Answer: Title: Classic Margherita Pizza
Ingredients:
- 2 pkg. pizza dough mix
- 1 jar marinara sauce
- Fresh basil leaves
- 1 ball buffalo mozzarella cheese
- Extra virgin olive oil
- Salt to taste

Directions:
1. Preheat your oven to its highest setting, typically around 475°F (246°C). If you have a pizza stone, put it in the oven now too.
2. On a lightly floured surface, stretch or roll out one package of pizza dough into an approximately 12-inch circle. Transfer the dough onto a piece of parchment paper if using a baking sheet; otherwise, transfer directly onto a preheated pizza stone.
3. Spread half of the marinara sauce evenly across the base of the pizza, leaving a border for the crust edge.
4. Slice the buffalo mozzarella into thin rounds and distribute them over the sauce. Add some torn fresh basil leaves along with a drizzle of extra virgin olive oil. Season with a pinch of salt.
5. Carefully slide the pizza (with the parchment paper underneath if used) on